In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master('local') \
    .appName('demo') \
    .config("spark.driver.memory", "2g") \
    .config('spark.executor.memory', '2g') \
    .getOrCreate()
    
sc = spark.sparkContext

24/04/27 16:37:10 WARN Utils: Your hostname, rst-MS-7D99 resolves to a loopback address: 127.0.1.1; using 192.168.50.2 instead (on interface enp4s0)
24/04/27 16:37:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/27 16:37:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.executor.memory', '2g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.host', 'DESKTOP-2K42TPL')

In [2]:
data_rdd = sc.textFile('./test.csv')
data_rdd.take(5)

['"기준_날짜","집계_기준","기준_시간대","시작_대여소_ID","시작_대여소명","종료_대여소_ID","종료_대여소명","전체_건수","전체_이용_분","전체_이용_거리"',
 '"20240421","출발시간","0000","ST-1041","성내1동_033_1","ST-495","성내3동_014_1","1","9","1752"',
 '"20240421","출발시간","0000","ST-1046","성내2동_015_1","ST-3209","천호2동_048_1","1","8","1000"',
 '"20240421","출발시간","0000","ST-105","성수2가3동_015_1","ST-987","자양3동_004_1","1","13","1880"',
 '"20240421","출발시간","0000","ST-1061","천호1동_039_1","ST-1626","천호3동_040_1","1","6","1344"']

In [3]:
from collections.abc import Iterable
from collections import defaultdict
from typing import List

def fiilterHead(seq:str):
    if seq.startswith('"기준'):
        return False
    return True


def extractRdd(iterator:Iterable[str]):
    hash_map = defaultdict(int)
    
    for seq in iterator:
        seq = seq.split(',')
        start, end, num = seq[3][1:-1], seq[5][1:-1], int(seq[7][1:-1])
        
        hash_map[start+','+end] += num
    
    result = [(k, v) for k, v in hash_map.items()]
    return result

def extractMatrixA(iterator:Iterable[tuple[str, int]]):
    
    hash_map = defaultdict(List)
    
    for tup in iterator:
        start, end = tup[0].split(',')
        num = tup[1]
        
        if start not in hash_map:
            hash_map[start] = []
            
        hash_map[start].append((end, num))
    
    result = [(end_node, num_sum) for end_node, num_sum in hash_map.items()]
    return result

def extractMatrixA_t(iterator:Iterable[tuple[str, int]]):
    
    hash_map = defaultdict(List)
    
    for tup in iterator:
        start, end = tup[0].split(',')
        num = tup[1]
        
        if end not in hash_map:
            hash_map[end] = []
            
        hash_map[end].append((start, num))
    
    result = [(start_node, num_sum) for start_node, num_sum in hash_map.items()]
    return result

    

rdd = data_rdd.filter(fiilterHead).mapPartitions(extractRdd)
matA = rdd.mapPartitions(extractMatrixA)
matA_t = rdd.mapPartitions(extractMatrixA_t)
## RDD is like below
## (start stop, [(end stop, number of bikes used that route), ....])
matA_t.take(25)


[('ST-495', [('ST-1041', 2), ('ST-1839', 3), ('ST-1061', 4)]),
 ('ST-3209', [('ST-1046', 2), ('ST-505', 2)]),
 ('ST-987', [('ST-105', 2)]),
 ('ST-1626', [('ST-1061', 2), ('ST-502', 2), ('ST-1834', 2)]),
 ('ST-2551', [('ST-1069', 2)]),
 ('ST-1684', [('ST-1083', 4), ('ST-496', 2), ('ST-1046', 1)]),
 ('ST-2740', [('ST-1083', 2), ('ST-2574', 1)]),
 ('ST-1671', [('ST-1098', 2), ('ST-2630', 2)]),
 ('ST-625', [('ST-1147', 2)]),
 ('ST-2847', [('ST-1184', 2)]),
 ('ST-1347', [('ST-1215', 2)]),
 ('ST-1065', [('ST-1251', 2), ('ST-1718', 2), ('ST-2832', 1)]),
 ('ST-1514', [('ST-1252', 2), ('ST-835', 2)]),
 ('ST-705', [('ST-1259', 2), ('ST-2873', 1), ('ST-664', 1)]),
 ('ST-2284', [('ST-1263', 2)]),
 ('ST-1467', [('ST-1297', 2), ('ST-1111', 1)]),
 ('ST-1300', [('ST-1300', 2), ('ST-3099', 2)]),
 ('ST-144', [('ST-132', 2)]),
 ('ST-2558', [('ST-1320', 2)]),
 ('ST-2449', [('ST-1325', 2)]),
 ('ST-437', [('ST-1325', 2)]),
 ('ST-1446', [('ST-1326', 2), ('ST-207', 1)]),
 ('ST-1259', [('ST-1337', 2), ('ST-668